# Four point functions in the Potts and $O(n)$ models

This notebook reproduces some known results about Potts and $O(n)$ four-point functions, using the new Julia code.

In [110]:
setprecision(BigFloat, 64)
Pkg.activate("..")
using BootstrapVirasoro, DoubleFloats

  Activating project at `~/Documents/Recherche/projet_these/code/BootstrapVirasoro`


## Spectra

We create functions to generate the spectra of the $O(n)$ and Potts CFTs, with a cutoff:

\begin{align}
\mathcal{S}^{O(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in 2\mathbb{N}+1} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \frac12\mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^{PSU(n)} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}^*\\ s\in\frac{1}{r}\mathbb{Z}}}  \ ,
 \\
 \mathcal{S}^\text{Potts} &= \left\{V^d_{\langle 1,s\rangle}\right\}_{s\in\mathbb{N}^*} \bigcup \left\{V_{(r,s)}\right\}_{\substack{r\in \mathbb{N}+2\\ s\in\frac{1}{r}\mathbb{Z}}} \bigcup  \left\{ V_{P_{(0,s)}}\right\}_{s\in \mathbb{N}+\frac12}\ .
\end{align}


In [112]:
# c = CentralCharge(β=1/(big"0.8" + big"0.1"*im))
c = CC(β=1/ComplexDF64(df64"0.8", df64"0.1"))
Δmax = 10.

indices = ((1//2, 0), (1//2, 0), (1, 0), (1, 0))

fields = [Field(c, r=r, s=s) for (r, s) in indices]
co = Correlation(fields..., Δmax=Δmax)

LoopFields = Dict(
    :On => vcat(
        [Field(c, r=r, s=s) for r in 1//2:1//2:50 for s in -1+1//r:1//r:1 if r*s%1 == 0],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:2:50]
    ),
    :PSUn => vcat(
        [Field(c, r=r, s=s) for r in 1:50 for s in -1+1//r:1//r:1],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:50]
    ),
    :Potts => vcat(
        [Field(c, r=r, s=s) for r in 2:50 for s in -1+1//r:1//r:1],
        [Field(c, r=0, s=s, diagonal=true) for s in 1//2:1:3//2],
        [Field(c, r=1, s=s, diagonal=true) for s in 1:50]
    )
)

LoopSpectra = Dict(
    model => Spectrum(LoopFields[model], Δmax, interchiral=true)
    for model in (:On, :PSUn, :Potts)
)

SOn = LoopSpectra[:On];
SOn

Degenerate:
<1, 1>
<1, 3>
<1, 5>
<1, 7>
Non-diagonal:
(1, 0), (1, 1)
(2, -1//2), (2, 0), (2, 1//2), (2, 1)
(3, -2//3), (3, -1//3), (3, 0), (3, 1//3), (3, 2//3), (3, 1)

## Bootstrap equations

We numerically solve

\begin{align}
\sum_{V \in \mathcal{S}^{(s)}} D^{(s)}_V \mathcal I^{(s)}_V (x) = \sum_{V \in \mathcal{S}^{(t)}} D^{(t)}_V \mathcal I^{(t)}_V (x) = \sum_{V \in \mathcal{S}^{(u)}} D^{(u)}_V \mathcal I^{(u)}_V (x),
\end{align}

for some channel spectra $\mathcal{S}^{(s)}, \mathcal{S}^{(t)}, \mathcal{S}^{(u)}$, where $\mathcal I$ are interchiral conformal blocks, and the $D$'s are four-point structure constants.

We solve this system as
\begin{equation}
\underbrace{
\begin{bmatrix}
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [-\mathcal I^{(t)}_{V_j}(x_i)]_{ij} & [0] \\
[\mathcal I^{(s)}_{V_j}(x_i)]_{ij} & [0] & [-\mathcal I^{(u)}_{V_j}(x_i)]_{ij}
\end{bmatrix}}_A
\begin{bmatrix}
[D^{(s)}_{V_j}]_j \\
[D^{(t)}_{V_j}]_j \\
[D^{(u)}_{V_j}]_j
\end{bmatrix} = 
\begin{bmatrix}
\sum_{V_j \in \text{ known}} D^{(t)}_{V_j} \mathcal{I}^{(t)}_{V_j}(x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)\\
\sum_{V_j \in \text{ known}} D^{(u)}_{V_j} \mathcal{I}^{(u)}_{V_j}(x_i) - \sum_{V_j \in \text{ known}} D^{(s)}_{V_j} \mathcal{I}^{(s)}_{V_j}(x_i)
\end{bmatrix}
\end{equation}
where the $x_i$ take more values than there are unknowns, i.e. $A$ is a tall rectangular matrix. 

To check numerical convergence, we solve two subsystems and compare the solutions. If the solutions are close, we know the computation has converged.

In [113]:
b = BootstrapSystem(Schan_On);
compute_linear_system!(b);
solve!(b)
b.consts

Channel s
Fields        | Structure constants          | Errors                     
-------------------------------------------------------------------
<1, 1>        |  1.00000e+00 + 0.00000e+00im | 0.00000e+00 + 0.00000e+00im
<1, 3>        | -1.47107e-03 + 2.61048e-03im |  9.47561e-01 + 0.00000e+00im
<1, 5>        |  6.80347e-09 - 6.51724e-10im |  1.05080e+00 + 0.00000e+00im
<1, 7>        |  2.05501e-17 - 8.15714e-18im |  1.74517e+00 + 0.00000e+00im
V_{(2, 0)}    |  7.97271e-17 + 2.72250e-16im |  1.18687e+00 + 0.00000e+00im
V_{(2, 1//2)} |  1.18389e-16 - 1.49810e-16im |  1.20245e+00 + 0.00000e+00im
V_{(2, 1)}    | -7.41354e-17 + 1.37069e-17im |  1.18746e+00 + 0.00000e+00im
V_{(2, 3//2)} |  1.68664e-17 + 1.09953e-19im |  1.13402e+00 + 0.00000e+00im
V_{(3, 0)}    | -5.47386e-14 - 7.02483e-14im |  1.67373e+00 + 0.00000e+00im
V_{(3, 1//3)} |  1.10542e-14 + 8.34562e-14im |  1.81008e+00 + 0.00000e+00im
V_{(3, 2//3)} |  2.22284e-14 - 5.72438e-14im |  1.80211e+00 + 0.00000e+00im
V_{(3, 1)}  

In [114]:
b.matrix.LHS

52×45 Matrix{Complex{BigFloat}}:
   0.128098-0.254513im     1.02108e+15-4.29043e+16im  …       0.0+0.0im
   0.196034-0.730769im    -2.49391e+14-4.79099e+16im          0.0+0.0im
   0.182057-0.510856im     2.11377e+14-4.58668e+16im          0.0+0.0im
   0.166957-0.408419im     4.32477e+14-4.51817e+16im          0.0+0.0im
   0.083955-0.177574im     8.49211e+14-4.22715e+16im          0.0+0.0im
   0.152826-0.378053im     2.91241e+14-4.57896e+16im  …       0.0+0.0im
  0.0351539-0.0286636im    3.74852e+15-4.19858e+16im          0.0+0.0im
   0.114839-0.192248im     1.40962e+15-4.26402e+16im          0.0+0.0im
  0.0288052-0.0334415im    2.37357e+15-4.05415e+16im          0.0+0.0im
   0.198657-0.664329im    -2.12632e+14-4.77863e+16im          0.0+0.0im
  0.0452373-0.0468913im    2.88764e+15-4.19413e+16im  …       0.0+0.0im
  0.0788153-0.141888im     1.22702e+15-4.1663e+16im           0.0+0.0im
  0.0577913-0.0653575im    2.58247e+15-4.11804e+16im          0.0+0.0im
           ⋮                   